In [1]:
import pandas as pd
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read the csv files (Movies and Ratings)
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [3]:
# separate the year of each movies from title column and ctrate a new column for years (just numbers without any Braces)
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))', expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)', expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

C:\Users\MT\AppData\Local\Temp/ipykernel_7824/522903812.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [4]:
# Split each genre with "|" and make sth like a list of genres
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [5]:
movies_with_genres_df = movies_df.copy() # Copy the movies DataFrame and create a new one like that

for index, row in movies_with_genres_df.iterrows(): # for every row in Movie DataFrame , if the genres are in the movie replace it to 1 
    for genre in row['genres']:
        movies_with_genres_df.at[index, genre] = 1
        
        
# At the end write 0 for that genres which are not for that movie.(It is a little hard to explain it in english :) )   
movies_with_genres_df = movies_with_genres_df.fillna(0) 
movies_with_genres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# In rating csv , remove timestamp column
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

C:\Users\MT\AppData\Local\Temp/ipykernel_7824/1107861065.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ratings_df = ratings_df.drop('timestamp', 1)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [7]:
# Create a user profile of watched movies that contains the name and rate of each movies
userInput = [
    {'title':'12 Angry Men', 'rating':5},
    {'title':'There Will Be Blood', 'rating':4},
    {'title':'Papillon', 'rating':4},
    {'title':'The Silence of the Lambs', 'rating':3},
    {'title':'Enemy of the State', 'rating':3}
    
]

# Create a DataFrame of userInput (That contains the name and ratings of movies that have watched)
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,12 Angry Men,5
1,There Will Be Blood,4
2,Papillon,4
3,The Silence of the Lambs,3
4,Enemy of the State,3


In [8]:
# If the title of movies is existes in inputMovies , create a list of them
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

inputMovies = pd.merge(inputId, inputMovies) # merge the InputId and InputMovies together

inputMovies = inputMovies.drop('genres',1).drop('year', 1) # drop genres and year columns

inputMovies

C:\Users\MT\AppData\Local\Temp/ipykernel_7824/3051924602.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  inputMovies = inputMovies.drop('genres',1).drop('year', 1) # drop genres and year columns


,movieId,title,rating
0,593,The Silence of the Lambs,3
1,1203,12 Angry Men,5
2,77846,12 Angry Men,5
3,2353,Enemy of the State,3
4,3198,Papillon,4
5,56782,There Will Be Blood,4


In [9]:
# If the movieId existes in InputMovies , Then create a list of them (A kind of filtering)
userMovies = movies_with_genres_df[movies_with_genres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
510,593,The Silence of the Lambs,"[Crime, Horror, Thriller]",1991,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
905,1203,12 Angry Men,[Drama],1957,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1755,2353,Enemy of the State,"[Action, Thriller]",1998,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2409,3198,Papillon,"[Crime, Drama]",1973,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6648,56782,There Will Be Blood,"[Drama, Western]",2007,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7332,77846,12 Angry Men,"[Crime, Drama]",1997,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
userMovies = userMovies.reset_index(drop=True) # reset the number of index and start at the first

# remove movieId, genres, title, and year columns
userGenreTable = userMovies.drop('movieId',1).drop('title',1).drop('genres',1).drop('year',1)
userGenreTable

C:\Users\MT\AppData\Local\Temp/ipykernel_7824/2113410367.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  userGenreTable = userMovies.drop('movieId',1).drop('title',1).drop('genres',1).drop('year',1)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# To get user profile we have to DOt usetGenreTable to inputMovies :)
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile 

Adventure              0.0
Animation              0.0
Children               0.0
Comedy                 0.0
Fantasy                0.0
Romance                0.0
Drama                 16.0
Action                 5.0
Crime                 10.0
Thriller               8.0
Horror                 3.0
Mystery                0.0
Sci-Fi                 0.0
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                4.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [12]:
# from Movies with genres just get the genres of movies and write them to genreTable
genreTable = movies_with_genres_df.set_index(movies_with_genres_df['movieId'])

# Remove the unnecessary columns
genreTable = genreTable.drop('movieId',1).drop('title',1).drop('genres', 1).drop('year',1)
genreTable.head()

C:\Users\MT\AppData\Local\Temp/ipykernel_7824/3557667000.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  genreTable = genreTable.drop('movieId',1).drop('title',1).drop('genres', 1).drop('year',1)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# to calculate the weighted average we have to multiply the genres with weights
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df

movieId
1         0.000000
2         0.000000
3         0.000000
4         0.347826
5         0.000000
            ...   
193581    0.108696
193583    0.000000
193585    0.347826
193587    0.108696
193609    0.000000
Length: 9742, dtype: float64

In [14]:
# Sort the values and at the first show the best values...
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

movieId
81132    1.000000
4210     0.913043
7235     0.913043
55721    0.847826
79132    0.847826
dtype: float64

In [15]:
# if movieId is in recommendation table the show 20 of the (20 movies for recommend to user)
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
70,78,"Crossing Guard, The","[Action, Crime, Drama, Thriller]",1995
454,519,RoboCop 3,"[Action, Crime, Drama, Sci-Fi, Thriller]",1993
2744,3682,Magnum Force,"[Action, Crime, Drama, Thriller]",1973
3128,4210,Manhunter,"[Action, Crime, Drama, Horror, Thriller]",1986
3657,5027,Another 48 Hrs.,"[Action, Comedy, Crime, Drama, Thriller]",1990
4249,6196,Cradle 2 the Grave,"[Action, Crime, Drama, Thriller]",2003
4843,7235,Ichi the Killer (Koroshiya 1),"[Action, Comedy, Crime, Drama, Horror, Thriller]",2001
5088,8016,"Getaway, The","[Action, Crime, Drama, Thriller]",1972
5305,8798,Collateral,"[Action, Crime, Drama, Thriller]",2004
5327,8860,Cellular,"[Action, Crime, Drama, Mystery, Thriller]",2004
